<a href="https://colab.research.google.com/github/LaTsa99/DeepMinds/blob/master/DeepMinds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Importing necessary modules for data preprocessing**

In [1]:
import os
import zipfile
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

**Downloading the zip file containing our training data**

In [2]:
!pip install gdown
!gdown https://drive.google.com/uc?id=1UM65-NZoXAr3lydSkmX2OmHm63-PfepX -O /tmp/girls.zip

Downloading...
From: https://drive.google.com/uc?id=1UM65-NZoXAr3lydSkmX2OmHm63-PfepX
To: /tmp/girls.zip
126MB [00:01, 89.0MB/s]


**Unzipping the zip file containing the training data**

In [22]:
local_zip = '/tmp/girls.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

**Creating path strings to easily import the train data**

In [23]:
base_dir = os.path.join('/tmp', 'girls')

train_dir = os.path.join(base_dir, 'train')
train_bikini_dir = os.path.join(train_dir, 'bikini')
train_long_dir = os.path.join(train_dir, 'long')

valid_dir = os.path.join(base_dir, 'valid')
valid_bikini_dir = os.path.join(valid_dir, 'bikini')
valid_long_dir = os.path.join(valid_dir, 'long')

test_dir = os.path.join(base_dir, 'test')
test_bikini_dir = os.path.join(test_dir, 'bikini')
test_long_dir = os.path.join(test_dir, 'long')

**Querrying the filenames into variables**

In [24]:
bikini_train_fnames = os.listdir(train_bikini_dir)
bikini_valid_fnames = os.listdir(valid_bikini_dir)
bikini_test_fnames = os.listdir(test_bikini_dir)

long_train_fnames = os.listdir(train_long_dir)
long_valid_fnames = os.listdir(valid_long_dir)
long_test_fnames = os.listdir(test_long_dir)

**Setting the input image dimensions**

In [25]:
img_height = 299
img_width = 299

**Generating our data and making data generators**

In [26]:
# TODO: preproc fv-be

# TODO: adatdúsítás a train-nél
train_datagen = ImageDataGenerator(rescale=1./255)
valid_datagen = ImageDataGenerator(rescale=1./255)
test_datagen  = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir, target_size=(img_height, img_width), batch_size=20, class_mode='binary')
valid_generator = valid_datagen.flow_from_directory(valid_dir, target_size=(img_height, img_width), batch_size=20, class_mode='binary')
test_generator  = test_datagen.flow_from_directory(test_dir, target_size=(img_height, img_width), batch_size=20, class_mode='binary')

Found 1572 images belonging to 2 classes.
Found 527 images belonging to 2 classes.
Found 266 images belonging to 2 classes.


In [27]:
from tensorflow.keras.layers import Dense, Conv2D, GlobalMaxPooling2D, Dropout, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input, decode_predictions

In [28]:
base_model = InceptionV3(weights='imagenet', include_top=False)

In [29]:
x = base_model.output
x = GlobalMaxPooling2D()(x)
x = Dense(1024, activation='relu')(x)

# TODO: további rétegek
# TODO: ragularizációk

predictions = Dense(1, activation='sigmoid')(x)
model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
  layer.trainable = False

# TODO: rmsprop, ...

model.compile(optimizer='adam', metrics=['accuracy'], loss='binary_crossentropy')

In [34]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard

early_stopping = EarlyStopping(patience=10, verbose=1, min_delta=0.01)
checkpointer = ModelCheckpoint(filepath='model.hdf5', save_best_only=True, verbose=1)
tb = TensorBoard(log_dir='logs', histogram_freq=1, write_graph=1)

In [35]:
model.fit_generator(train_generator, steps_per_epoch=79, validation_data=valid_generator, validation_steps=1, epochs=1, callbacks=[early_stopping, checkpointer, tb])

 1/79 [..............................] - ETA: 0s - loss: 0.0348 - accuracy: 0.9500WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
79/79 [==============================] - ETA: 0s - loss: 0.2103 - accuracy: 0.9656
Epoch 00001: val_loss improved from inf to 0.06179, saving model to model.hdf5
79/79 [==============================] - 332s 4s/step - loss: 0.2103 - accuracy: 0.9656 - val_loss: 0.0618 - val_accuracy: 0.9500


In [ ]:
# TODO: kifagyasztás, újratanítás

In [39]:
from keras.models import load_model

model = load_model('model.hdf5')
test_err = model.evaluate_generator(test_generator)
print('Test error:', test_err[0], 'Test accuracy:', test_err[1])

Instructions for updating:
Please use Model.evaluate, which supports generators.
Test error: 0.32037538290023804 Test accuracy: 0.9473684430122375


In [48]:
from tensorflow.keras.preprocessing import image
import numpy as np

# TODO

img_path = 'test3.jpeg'
test0  = image.load_img(img_path, target_size=(img_height, img_width))
x = image.img_to_array(test0)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

pred = model.predict(x)
print(pred)

[[2.400311e-05]]


In [ ]:
# TODO: tensorboard